# プロンプトの最適化

AIモデルを使用する際、効果的なプロンプトを作成することは重要なスキルです。経験豊富なユーザーでも、うっかり矛盾、曖昧さ、または一貫性の欠如を導入してしまい、最適でない結果につながることがあります。ここで紹介するシステムは、一般的な問題を特定して修正し、より信頼性が高く効果的なプロンプトを作成するのに役立ちます。

最適化プロセスでは、特化したAIエージェントが協力してプロンプトを分析し、書き直すマルチエージェントアプローチを使用します。システムは自動的に以下のような一般的な問題の種類を特定し、対処します：

- プロンプト指示における**矛盾**
- 不足または不明確な**フォーマット仕様**
- プロンプトとfew-shot例の間の**一貫性の欠如**

---

**目的**: このクックブックでは、Agents SDKとEvalsを組み合わせて使用し、OpenAIのプロンプト最適化システムの初期バージョンを構築するためのベストプラクティスを紹介します。このコードを使用してプロンプトを最適化するか、[プレイグラウンドのオプティマイザー](https://platform.openai.com/playground/prompts)を使用することができます！

**クックブックの構成**  
このノートブックは以下の構成に従っています：

- [ステップ1. システム概要](#1-system-overview) - プロンプト最適化システムの仕組みを学ぶ  
- [ステップ2. データモデル](#2-data-models) - システムで使用されるデータ構造を理解する
- [ステップ3. エージェントの定義](#3-defining-the-agents) - プロンプトを分析・改善するエージェントを見る
- [ステップ4. 評価](#4-using-evaluations-to-arrive-at-these-agents) - Evalsを使用してエージェントモデルの選択と指示を検証する
- [ステップ5. 最適化ワークフローの実行](#4-run-optimization-workflow) - ワークフローがプロンプトを引き渡す方法を見る
- [ステップ6. 例](#5-examples) - プロンプト最適化の実際の例を探る

**前提条件**
- `openai` Pythonパッケージ 
- `openai-agents` パッケージ
- 環境変数として`OPENAI_API_KEY`に設定されたOpenAI APIキー

## 1. システム概要

プロンプト最適化システムは、協調的なマルチエージェントアプローチを使用してプロンプトを分析し、改善します。各エージェントは、特定のタイプの問題の検出または書き直しに特化しています：

1. **Dev-Contradiction-Checker**: プロンプト内の論理的矛盾や不可能な指示をスキャンします。例えば、同じプロンプト内で「正の数のみを使用する」と「負の例を含める」といった矛盾を検出します。

2. **Format-Checker**: プロンプトが構造化された出力（JSON、CSV、Markdownなど）を期待しているにも関わらず、正確なフォーマット要件を明確に指定していない場合を特定します。このエージェントは、必要なすべてのフィールド、データ型、フォーマットルールが明示的に定義されていることを確認します。

3. **Few-Shot-Consistency-Checker**: 例示会話を検査して、アシスタントの応答がプロンプトで指定されたルールに実際に従っていることを確認します。これにより、プロンプトが要求する内容と例が示す内容の不一致を捉えます。

4. **Dev-Rewriter**: 問題が特定された後、このエージェントは元の意図を保持しながら、矛盾を解決し、フォーマット仕様を明確化するためにプロンプトを書き直します。

5. **Few-Shot-Rewriter**: 一貫性のない例示応答を更新して、プロンプト内のルールと整合させ、すべての例が新しい開発者プロンプトに適切に準拠することを確保します。

これらのエージェントが連携することで、プロンプトの問題を体系的に特定し、修正することができます。

In [ ]:
# Import required modules
from openai import AsyncOpenAI
import asyncio
import json
import os
from enum import Enum
from typing import Any, List, Dict
from pydantic import BaseModel, Field
from agents import Agent, Runner, set_default_openai_client, trace

openai_client: AsyncOpenAI | None = None

def _get_openai_client() -> AsyncOpenAI:
    global openai_client
    if openai_client is None:
        openai_client = AsyncOpenAI(
            api_key=os.environ.get("OPENAI_API_KEY", "your-api-key"),
        )
    return openai_client

set_default_openai_client(_get_openai_client())

## 2. データモデル

エージェント間の構造化された通信を促進するため、システムはPydanticモデルを使用して入力と出力の期待される形式を定義しています。これらのPydanticモデルは、データの検証を支援し、ワークフロー全体を通じて一貫性を確保します。

データモデルには以下が含まれます：

1. **Role** - メッセージの役割（user/assistant）の列挙型
2. **ChatMessage** - 会話内の単一メッセージを表現
3. **Issues** - 検出された問題を報告するためのベースモデル
4. **FewShotIssues** - サンプルメッセージの書き直し提案を追加した拡張モデル
5. **MessagesOutput** - 最適化された会話メッセージを含む
6. **DevRewriteOutput** - 改善された開発者プロンプトを含む

Pydanticを使用することで、システムはプロセスの各ステップで、すべてのデータが期待される形式に準拠していることを検証できます。

In [32]:
class Role(str, Enum):
    """Role enum for chat messages."""
    user = "user"
    assistant = "assistant"

class ChatMessage(BaseModel):
    """Single chat message used in few-shot examples."""
    role: Role
    content: str

class Issues(BaseModel):
    """Structured output returned by checkers."""
    has_issues: bool
    issues: List[str]
    
    @classmethod
    def no_issues(cls) -> "Issues":
        return cls(has_issues=False, issues=[])

class FewShotIssues(Issues):
    """Output for few-shot contradiction detector including optional rewrite suggestions."""
    rewrite_suggestions: List[str] = Field(default_factory=list)
    
    @classmethod
    def no_issues(cls) -> "FewShotIssues":
        return cls(has_issues=False, issues=[], rewrite_suggestions=[])

class MessagesOutput(BaseModel):
    """Structured output returned by `rewrite_messages_agent`."""

    messages: list[ChatMessage]


class DevRewriteOutput(BaseModel):
    """Rewriter returns the cleaned-up developer prompt."""

    new_developer_message: str

## 3. エージェントの定義

このセクションでは、`openai-agents`パッケージの`Agent`クラスを使用して、特化したAIエージェントを作成します。これらのエージェント定義を見ると、効果的なAI指示を作成するためのいくつかのベストプラクティスが明らかになります：

### エージェント指示のベストプラクティス

1. **明確なスコープ定義**: 各エージェントは明確な境界を持つ狭く定義された目的を持っています。例えば、矛盾チェッカーは「真の自己矛盾」のみに焦点を当て、「重複や冗長性は矛盾ではない」と明示的に述べています。

2. **段階的なプロセス**: 指示は明確な方法論を提供します。例えば、フォーマットチェッカーがフォーマット要件を分析する前に、まずタスクを分類する方法などです。

3. **明示的な定義**: 曖昧さを避けるために、重要な用語が正確に定義されています。few-shot一貫性チェッカーには、コンプライアンスが何を構成するかを正確に説明する詳細な「コンプライアンス評価基準」が含まれています。

4. **境界設定**: 指示はエージェントが何をすべきでないかを明記しています。few-shotチェッカーは、問題の過度な指摘を防ぐために「スコープ外」のものを明示的にリストアップしています。

5. **構造化された出力要件**: 各エージェントは例を含む厳密に定義された出力フォーマットを持ち、最適化パイプラインでの一貫性を確保しています。

これらの原則により、最適化システムで効果的に連携する信頼性の高い、焦点を絞ったエージェントが作成されます。以下では、詳細な指示を含む完全なエージェント定義を見ていきます。

In [ ]:
dev_contradiction_checker = Agent(
    name="contradiction_detector",
    model="gpt-4.1",
    output_type=Issues,
    instructions="""
    You are **Dev-Contradiction-Checker**.

    Goal
    Detect *genuine* self-contradictions or impossibilities **inside** the developer prompt supplied in the variable `DEVELOPER_MESSAGE`.

    Definitions
    • A contradiction = two clauses that cannot both be followed.
    • Overlaps or redundancies in the DEVELOPER_MESSAGE are *not* contradictions.

    What you MUST do
    1. Compare every imperative / prohibition against all others.
    2. List at most FIVE contradictions (each as ONE bullet).
    3. If no contradiction exists, say so.

    Output format (**strict JSON**)
    Return **only** an object that matches the `Issues` schema:

    ```json
    {"has_issues": <bool>,
    "issues": [
        "<bullet 1>",
        "<bullet 2>"
    ]
    }
    - has_issues = true IFF the issues array is non-empty.
    - Do not add extra keys, comments or markdown.
""",
)
format_checker = Agent(
    name="format_checker",
    model="gpt-4.1",
    output_type=Issues,
    instructions="""
    You are Format-Checker.

    Task
    Decide whether the developer prompt requires a structured output (JSON/CSV/XML/Markdown table, etc.).
    If so, flag any missing or unclear aspects of that format.

    Steps
    Categorise the task as:
    a. "conversation_only", or
    b. "structured_output_required".

    For case (b):
    - Point out absent fields, ambiguous data types, unspecified ordering, or missing error-handling.

    Do NOT invent issues if unsure. be a little bit more conservative in flagging format issues

    Output format
    Return strictly-valid JSON following the Issues schema:

    {
    "has_issues": <bool>,
    "issues": ["<desc 1>", "..."]
    }
    Maximum five issues. No extra keys or text.
""",
)
fewshot_consistency_checker = Agent(
    name="fewshot_consistency_checker",
    model="gpt-4.1",
    output_type=FewShotIssues,
    instructions="""
    You are FewShot-Consistency-Checker.

    Goal
    Find conflicts between the DEVELOPER_MESSAGE rules and the accompanying **assistant** examples.

    USER_EXAMPLES:      <all user lines>          # context only
    ASSISTANT_EXAMPLES: <all assistant lines>     # to be evaluated

    Method
    Extract key constraints from DEVELOPER_MESSAGE:
    - Tone / style
    - Forbidden or mandated content
    - Output format requirements

    Compliance Rubric - read carefully
    Evaluate only what the developer message makes explicit.

    Objective constraints you must check when present:
    - Required output type syntax (e.g., "JSON object", "single sentence", "subject line").
    - Hard limits (length ≤ N chars, language required to be English, forbidden words, etc.).
    - Mandatory tokens or fields the developer explicitly names.

    Out-of-scope (DO NOT FLAG):
    - Whether the reply "sounds generic", "repeats the prompt", or "fully reflects the user's request" - unless the developer text explicitly demands those qualities.
    - Creative style, marketing quality, or depth of content unless stated.
    - Minor stylistic choices (capitalisation, punctuation) that do not violate an explicit rule.

    Pass/Fail rule
    - If an assistant reply satisfies all objective constraints, it is compliant, even if you personally find it bland or loosely related.
    - Only record an issue when a concrete, quoted rule is broken.

    Empty assistant list ⇒ immediately return has_issues=false.

    For each assistant example:
    - USER_EXAMPLES are for context only; never use them to judge compliance.
    - Judge each assistant reply solely against the explicit constraints you extracted from the developer message.
    - If a reply breaks a specific, quoted rule, add a line explaining which rule it breaks.
    - Optionally, suggest a rewrite in one short sentence (add to rewrite_suggestions).
    - If you are uncertain, do not flag an issue.
    - Be conservative—uncertain or ambiguous cases are not issues.

    be a little bit more conservative in flagging few shot contradiction issues
    Output format
    Return JSON matching FewShotIssues:

    {
    "has_issues": <bool>,
    "issues": ["<explanation 1>", "..."],
    "rewrite_suggestions": ["<suggestion 1>", "..."] // may be []
    }
    List max five items for both arrays.
    Provide empty arrays when none.
    No markdown, no extra keys.
    """,
)
dev_rewriter = Agent(
    name="dev_rewriter",
    model="gpt-4.1",
    output_type=DevRewriteOutput,
    instructions="""
    You are Dev-Rewriter.

    You receive:
    - ORIGINAL_DEVELOPER_MESSAGE
    - CONTRADICTION_ISSUES (may be empty)
    - FORMAT_ISSUES (may be empty)

    Rewrite rules
    Preserve the original intent and capabilities.

    Resolve each contradiction:
    - Keep the clause that preserves the message intent; remove/merge the conflicting one.

    If FORMAT_ISSUES is non-empty:
    - Append a new section titled ## Output Format that clearly defines the schema or gives an explicit example.

    Do NOT change few-shot examples.

    Do NOT add new policies or scope.

    Output format (strict JSON)
    {
    "new_developer_message": "<full rewritten text>"
    }
    No other keys, no markdown.
""",
)
fewshot_rewriter = Agent(
    name="fewshot_rewriter",
    model="gpt-4.1",
    output_type=MessagesOutput,
    instructions="""
    You are FewShot-Rewriter.

    Input payload
    - NEW_DEVELOPER_MESSAGE (already optimized)
    - ORIGINAL_MESSAGES (list of user/assistant dicts)
    - FEW_SHOT_ISSUES (non-empty)

    Task
    Regenerate only the assistant parts that were flagged.
    User messages must remain identical.
    Every regenerated assistant reply MUST comply with NEW_DEVELOPER_MESSAGE.

    After regenerating each assistant reply, verify:
    - It matches NEW_DEVELOPER_MESSAGE. ENSURE THAT THIS IS TRUE.

    Output format
    Return strict JSON that matches the MessagesOutput schema:

    {
    "messages": [
        {"role": "user", "content": "..."},
        {"role": "assistant", "content": "..."}
        ]
    }
    Guidelines
    - Preserve original ordering and total count.
    - If a message was unproblematic, copy it unchanged.
    """,
)

## 4. エージェントの構築における評価の活用

OpenAI Evalsを使用してエージェントの指示を調整し、適切なモデルを選択した方法を見てみましょう。そのために、ゴールデンサンプルのセットを構築しました。各サンプルには、元のメッセージ（開発者メッセージ + ユーザー/アシスタントメッセージ）と、最適化ワークフローが行うべき変更が含まれています。以下は、使用したゴールデンペアの2つの例です：

```
[
  {
    "focus": "contradiction_issues",
    "input_payload": {
      "developer_message": "Always answer in **English**.\nNunca respondas en inglés.",
      "messages": [
        {
          "role": "user",
          "content": "¿Qué hora es?"
        }
      ]
    },
    "golden_output": {
      "changes": true,
      "new_developer_message": "Always answer **in English**.",
      "new_messages": [
        {
          "role": "user",
          "content": "¿Qué hora es?"
        }
      ],
      "contradiction_issues": "開発者メッセージが同時に英語を要求し、それを禁止している。",
      "few_shot_contradiction_issues": "",
      "format_issues": "",
      "general_improvements": ""
    }
  },
  {
    "focus": "few_shot_contradiction_issues",
    "input_payload": {
      "developer_message": "Respond with **only 'yes' or 'no'** – no explanations.",
      "messages": [
        {
          "role": "user",
          "content": "Is the sky blue?"
        },
        {
          "role": "assistant",
          "content": "Yes, because wavelengths …"
        },
        {
          "role": "user",
          "content": "Is water wet?"
        },
        {
          "role": "assistant",
          "content": "Yes."
        }
      ]
    },
    "golden_output": {
      "changes": true,
      "new_developer_message": "Respond with **only** the single word \"yes\" or \"no\".",
      "new_messages": [
        {
          "role": "user",
          "content": "Is the sky blue?"
        },
        {
          "role": "assistant",
          "content": "yes"
        },
        {
          "role": "user",
          "content": "Is water wet?"
        },
        {
          "role": "assistant",
          "content": "yes"
        }
      ],
      "contradiction_issues": "",
      "few_shot_contradiction_issues": "アシスタントの例には、説明しないという指示にもかかわらず説明が含まれている。",
      "format_issues": "",
      "general_improvements": ""
    }
  }
]
```

これらの20個の手動ラベル付けされたゴールデン出力から、矛盾の問題、few shotの問題、フォーマットの問題、問題なし、またはこれらの問題の組み合わせをカバーする範囲で、2つのことを検証するためのPython文字列チェック採点器を構築しました：各ゴールデンペアに対して問題が検出されたかどうか、および検出された問題が期待されたものと一致したかどうかです。この信号から、エージェントの指示と使用するモデルを調整して、この評価における精度を最大化しました。精度、コスト、速度のバランスを考慮して、4.1モデルに決定しました。使用した具体的なプロンプトも4.1プロンプティングガイドに従っています。ご覧のように、20個すべてのゴールデン出力で正しいラベルを達成しています：適切な問題を特定し、偽陽性を回避しています。

![ゴールデンセットの精度](../images/optimizepromptfig1.png)

![ゴールデンセットの評価](../images/optimizepromptfig2.png)

## 5. 最適化ワークフローの実行

最適化システムが実際にエンドツーエンドでどのように動作するかを詳しく見てみましょう。コアワークフローは、プロンプトを効率的に処理・最適化するために、エージェントを並列で複数回実行することで構成されています。

In [ ]:
def _normalize_messages(messages: List[Any]) -> List[Dict[str, str]]:
    """Convert list of pydantic message models to JSON-serializable dicts."""
    result = []
    for m in messages:
        if hasattr(m, "model_dump"):
            result.append(m.model_dump())
        elif isinstance(m, dict) and "role" in m and "content" in m:
            result.append({"role": str(m["role"]), "content": str(m["content"])})
    return result

async def optimize_prompt_parallel(
    developer_message: str,
    messages: List["ChatMessage"],
) -> Dict[str, Any]:
    """
    Runs contradiction, format, and few-shot checkers in parallel,
    then rewrites the prompt/examples if needed.
    Returns a unified dict suitable for an API or endpoint.
    """

    with trace("optimize_prompt_workflow"):
        # 1. Run all checkers in parallel (contradiction, format, fewshot if there are examples)
        tasks = [
            Runner.run(dev_contradiction_checker, developer_message),
            Runner.run(format_checker, developer_message),
        ]
        if messages:
            fs_input = {
                "DEVELOPER_MESSAGE": developer_message,
                "USER_EXAMPLES": [m.content for m in messages if m.role == "user"],
                "ASSISTANT_EXAMPLES": [m.content for m in messages if m.role == "assistant"],
            }
            tasks.append(Runner.run(fewshot_consistency_checker, json.dumps(fs_input)))

        results = await asyncio.gather(*tasks)

        # Unpack results
        cd_issues: Issues = results[0].final_output
        fi_issues: Issues = results[1].final_output
        fs_issues: FewShotIssues = results[2].final_output if messages else FewShotIssues.no_issues()

        # 3. Rewrites as needed
        final_prompt = developer_message
        if cd_issues.has_issues or fi_issues.has_issues:
            pr_input = {
                "ORIGINAL_DEVELOPER_MESSAGE": developer_message,
                "CONTRADICTION_ISSUES": cd_issues.model_dump(),
                "FORMAT_ISSUES": fi_issues.model_dump(),
            }
            pr_res = await Runner.run(dev_rewriter, json.dumps(pr_input))
            final_prompt = pr_res.final_output.new_developer_message

        final_messages: list[ChatMessage] | list[dict[str, str]] = messages
        if fs_issues.has_issues:
            mr_input = {
                "NEW_DEVELOPER_MESSAGE": final_prompt,
                "ORIGINAL_MESSAGES": _normalize_messages(messages),
                "FEW_SHOT_ISSUES": fs_issues.model_dump(),
            }
            mr_res = await Runner.run(fewshot_rewriter, json.dumps(mr_input))
            final_messages = mr_res.final_output.messages

        return {
            "changes": True,
            "new_developer_message": final_prompt,
            "new_messages": _normalize_messages(final_messages),
            "contradiction_issues": "\n".join(cd_issues.issues),
            "few_shot_contradiction_issues": "\n".join(fs_issues.issues),
            "format_issues": "\n".join(fi_issues.issues),
        }

![ワークフローのトレース](../images/optimizepromptfig3.png)

### 最適化ワークフローの理解

`optimize_prompt_parallel`関数は、並列化を通じて効率を最大化するワークフローを実装しています：

1. **並列問題検出**: 最初のフェーズでは、すべてのチェッカーエージェントが同時に実行されます：
   - `dev_contradiction_checker`はプロンプト内の論理的矛盾を検索します
   - `format_checker`は不明確なフォーマット仕様を探します
   - `fewshot_consistency_checker`（例が存在する場合）はプロンプトと例の間の不一致をチェックします

並列チェックフェーズの後、ワークフローは依存関係を慎重に処理します：

2. **プロンプト書き換え（条件付き）**: `dev_rewriter`エージェントは、矛盾またはフォーマットの問題が検出された場合のみ実行されます。このエージェントは以下の出力に依存します：
   - `dev_contradiction_checker`（`cd_issues`変数）
   - `format_checker`（`fi_issues`変数）

3. **例の書き換え（条件付き）**: `fewshot_rewriter`エージェントは、例の不整合が検出された場合のみ実行されます。このエージェントは以下に依存します：
   - 書き換えられたプロンプト（プロンプト書き換え後に実行される必要があります）
   - 元のメッセージ
   - Few-shotの問題（`fs_issues`変数）

## 6. 例

実際の例を使って最適化システムの動作を見てみましょう。

### 例1: 矛盾の修正

In [28]:
async def example_contradiction():
    # A prompt with contradictory instructions
    prompt = """Quick-Start Card — Product Parser

Goal  
Digest raw HTML of an e-commerce product detail page and emit **concise, minified JSON** describing the item.

**Required fields:**  
name | brand | sku | price.value | price.currency | images[] | sizes[] | materials[] | care_instructions | features[]

**Extraction priority:**  
1. schema.org/JSON-LD blocks  
2. <meta> & microdata tags  
3. Visible DOM fallback (class hints: "product-name", "price")

** Rules:**  
- If *any* required field is missing, short-circuit with: `{"error": "FIELD_MISSING:<field>"}`.
- Prices: Numeric with dot decimal; strip non-digits (e.g., "1.299,00 EUR" → 1299.00 + "EUR").
- Deduplicate images differing only by query string. Keep ≤10 best-res.
- Sizes: Ensure unit tag ("EU", "US") and ascending sort.
- Materials: Title-case and collapse synonyms (e.g., "polyester 100%" → "Polyester").

**Sample skeleton (minified):**
```json
{"name":"","brand":"","sku":"","price":{"value":0,"currency":"USD"},"images":[""],"sizes":[],"materials":[],"care_instructions":"","features":[]}
Note: It is acceptable to output null for any missing field instead of an error ###"""
    
    result = await optimize_prompt_parallel(prompt, [])
    
    # Display the results
    if result["contradiction_issues"]:
        print("Contradiction issues:")
        print(result["contradiction_issues"])
        print()
        
    print("Optimized prompt:")
    print(result["new_developer_message"])
    
# Run the example
await example_contradiction()

Contradiction issues:
There is a contradiction between the rule that says to short-circuit and output an error if *any* required field is missing ('{"error": "FIELD_MISSING:<field>"}') and the final note which states that it is acceptable to output null for any missing field instead of an error. Both behaviors cannot be followed simultaneously when a required field is missing.

Optimized prompt:
Quick-Start Card — Product Parser

Goal  
Digest raw HTML of an e-commerce product detail page and emit **concise, minified JSON** describing the item.

**Required fields:**  
name | brand | sku | price.value | price.currency | images[] | sizes[] | materials[] | care_instructions | features[]

**Extraction priority:**  
1. schema.org/JSON-LD blocks  
2. <meta> & microdata tags  
3. Visible DOM fallback (class hints: "product-name", "price")

**Rules:**  
- If *any* required field is missing, short-circuit and output: `{"error": "FIELD_MISSING:<field>"}`
- Prices: Numeric with dot decimal; strip

これは、システムが一貫性のない出力やモデルの混乱を引き起こす可能性のある重要な矛盾を検出し、解決する方法を示しています。

### 例2: プロンプトとFew-Shotサンプル間の不整合の修正

In [29]:
async def example_fewshot_fix():
    prompt = "Respond **only** with JSON using keys `city` (string) and `population` (integer)."
    
    messages = [
        {"role": "user", "content": "Largest US city?"},
        {"role": "assistant", "content": "New York City"},
        {"role": "user", "content": "Largest UK city?"},
        {"role": "assistant", "content": "{\"city\":\"London\",\"population\":9541000}"}
    ]
    
    
    print("Few-shot examples before optimization:")
    print(f"User: {messages[0]['content']}")
    print(f"Assistant: {messages[1]['content']}")
    print(f"User: {messages[2]['content']}")
    print(f"Assistant: {messages[3]['content']}")
    print()
    
    # Call the optimization API
    result = await optimize_prompt_parallel(prompt, [ChatMessage(**m) for m in messages])
    
    # Display the results
    if result["few_shot_contradiction_issues"]:
        print("Inconsistency found:", result["few_shot_contradiction_issues"])
        print()
    
    # Show the optimized few-shot examples
    optimized_messages = result["new_messages"]
    print("Few-shot examples after optimization:")
    print(f"User: {optimized_messages[0]['content']}")
    print(f"Assistant: {optimized_messages[1]['content']}")
    print(f"User: {optimized_messages[2]['content']}")
    print(f"Assistant: {optimized_messages[3]['content']}")
    
# Run the example
await example_fewshot_fix()

Few-shot examples before optimization:
User: Largest US city?
Assistant: New York City
User: Largest UK city?
Assistant: {"city":"London","population":9541000}

Inconsistency found: The response 'New York City' does not use JSON format and is missing the required keys `city` and `population` as stated in the rule 'Respond **only** with JSON using keys `city` (string) and `population` (integer).'

Few-shot examples after optimization:
User: Largest US city?
Assistant: {"city":"New York City","population":8468000}
User: Largest UK city?
Assistant: {"city":"London","population":9541000}


これは特に重要です。なぜなら、few-shotの例はモデルの応答方法に強い影響を与えるからです。例が記載されたルールに従っていない場合、モデルはそれらのルールを無視して例を模倣することを学習する可能性があります。プロンプトの指示と例の間の一貫性を確保することで、最適化システムはより信頼性の高いプロンプトを作成します。

### 例3: 長いプロンプトでの形式の明確化

In [30]:
async def example_format_issue():
    # A prompt with unclear or inconsistent formatting instructions
    prompt = """Task → Translate dense patent claims into 200-word lay summaries with a glossary.

Operating Steps:
1. Split the claim at semicolons, "wherein", or numbered sub-clauses.
2. For each chunk:
   a) Identify its purpose.
   b) Replace technical nouns with everyday analogies.
   c) Keep quantitative limits intact (e.g., "≥150 C").
3. Flag uncommon science terms with asterisks, and later define them.
4. Re-assemble into a flowing paragraph; do **not** broaden or narrow the claim’s scope.
5. Omit boilerplate if its removal does not alter legal meaning.

Output should follow a Markdown template:
- A summary section.
- A glossary section with the marked terms and their definitions.

Corner Cases:
- If the claim is over 5 kB, respond with CLAIM_TOO_LARGE.
- If claim text is already plain English, skip glossary and state no complex terms detected.

Remember: You are *not* providing legal advice—this is for internal comprehension only."""

    # Call the optimization API to check for format issues
    result = await optimize_prompt_parallel(prompt, [])

    # Display the results
    if result.get("format_issues"):
        print("Format issues found:", result["format_issues"])
        print()

    print("Optimized prompt:")
    print(result["new_developer_message"])

# Run the example
await example_format_issue()

Format issues found: Output must follow a precise Markdown template, but the expected structure (headers, formatting) for the summary and glossary sections is not fully specified.
Ambiguity if output should be a Markdown string or a structured object containing Markdown—data type of output is implicit.
No explicit ordering instruction for the summary and glossary sections—potentially ambiguous.
Word count limit (200 words) is mentioned for the summary but not for the glossary section—scope unclear.
No specific format for CLAIM_TOO_LARGE error or for indicating 'no complex terms'—should these be Markdown or plaintext?

Optimized prompt:
Task → Translate dense patent claims into 200-word lay summaries with a glossary.

Operating Steps:
1. Split the claim at semicolons, "wherein", or numbered sub-clauses.
2. For each chunk:
   a) Identify its purpose.
   b) Replace technical nouns with everyday analogies.
   c) Keep quantitative limits intact (e.g., ">=150 C").
3. Flag uncommon science te

この例では、フォーマットチェッカーが曖昧なフォーマット仕様をどのように識別し、解決するかを示しています。プロンプトはMarkdown出力を要求しており、最適化フローによってこれらのフォーマット仕様が大幅に改善されました。